# Using the Custom Vision Cognitive Servive
The *Custom Vision* cognitive service enables you to create a project in which you can train a classification model to identify images based on tags that you associate with them.

In this notebook, you will create and train a Custom Vision project that can identify pictures of apples and carrots, and use it to classify new images.

*Some of the images used in the lab are sourced from the free image library at <a href='http://www.pachd.com' target='_blank'>www.pachd.com</a>*

## Installing the Custom Vision SDK
The first step is to install the Python SDK for the Custom Vision service:

In [ ]:
# Install the Custom Vision SDK
! pip install azure-cognitiveservices-vision-customvision

## Sign up for a Custom Vision service account
Now you're ready to use the Custom Vision service. You'll need to sign up for an account and get your unique training and prediction keys so you can access it:
1. If you don't already have a Microsoft Azure subscription, sign up at https://azure.microsoft.com/en-us/. 
2. Go to https://customvision.ai/ and sign in using your Microsoft account. Then create a new custom vision service account in your Azure subscription.
3. Click the *Settings* (&#9881;) icon at the top right to view *training* and  *prediction* key's endpoints, and resource Ids. Then assign the appropriate values to the variables below and run the cell:

In [ ]:
TRAINING_KEY = 'YOUR_TRAINING_KEY'
PREDICTION_KEY = 'YOUR_PREDICTION_KEY'
ENDPOINT='https://YOUR_REGION.api.cognitive.microsoft.com' # Use just the base URL - https://<region>.api.cognitive.microsoft.com
PREDICTION_RESOURCE_ID="/subscriptions/YOUR_SUBSCRIPTION_ID/resourceGroups/YOUR_RESOURCE_GROUP/providers/Microsoft.CognitiveServices/accounts/YOUR_ACCOUNT_Prediction"

## Create a Custom Vision project
OK, now it's time to create a project for the apple/carrot identifier:

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient

trainer = CustomVisionTrainingClient(TRAINING_KEY, endpoint=ENDPOINT)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Produce Classification")
print("Created project!")

## Add tags
The project will identify images as apples or carrots, so we'll need tags for those classes:

In [ ]:
# Make two tags in the new project
print("Creating tags...")
apple_tag = trainer.create_tag(project.id, "Apple")
carrot_tag = trainer.create_tag(project.id, "Carrot")
print('Created tags!')

## Upload training images
Now that we've got the tags, we need to upload some images of apples and carrots, assign the appropriate tags:

In [ ]:
import os

print("Adding images...")

apples_dir = "data/apples"
for image in os.listdir(os.fsencode(apples_dir)):
    with open(apples_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data.read(), [apple_tag.id])

carrots_dir = "data/carrots"
for image in os.listdir(os.fsencode(carrots_dir)):
    with open(carrots_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data.read(), [carrot_tag.id])
        
print('Added images!')

Return to your Custom Vision service and click the *Home* (&#8962;) icon to return to the home page, and then open the ***Apple or Carrot*** project to view the images that have been uploaded and tagged.

## Train the project
With the tagged images in place, we're now ready to train the project:

In [ ]:
import time

print ("Training...")
# Train the project, checking status every 1 second
iteration = trainer.train_project(project.id)
while (iteration.status == "Training"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, "First Iteration", PREDICTION_RESOURCE_ID)

# Make it the default iteration
iteration = trainer.update_iteration(project_id= project.id, iteration_id=iteration.id, name= "First Iteration", is_default=True)

print ("Trained!")

Return to your Custom Vision service and click the **Performance** tab to view the *Precision* and *Recall* metrics for your trained project. These should be pretty high, even through we only used a few images.

## Use the project to classify images
Now that we have a trained project, we can use it to predict the class of new images that weren't in the training dataset:

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
%matplotlib inline

# Use two test images
test_img1_url = 'http://www.pachd.com/free-images/food-images/apple-01.jpg'
test_img2_url = 'http://www.pachd.com/free-images/food-images/carrot-02.jpg'

test_image_urls = []
test_image_urls.append(test_img1_url)
test_image_urls.append(test_img2_url)

# Create an instance of the prediction service
predictor = CustomVisionPredictionClient(PREDICTION_KEY, endpoint=ENDPOINT)

# Create a figure
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes
for url_idx in range(len(test_image_urls)):
    response = requests.get(test_image_urls[url_idx])
    image_contents = Image.open(BytesIO(response.content))
    results = predictor.classify_image_url(project_id=project.id, published_name=iteration.name, url=test_img_url)
    # The results include a prediction for each tag, in descending order of probability - so we'll get the first one
    prediction = results.predictions[0].tag_name + ": {0:.2f}%".format(results.predictions[0].probability * 100)
    # Subplot for image and its predicted class
    a=fig.add_subplot(1,2,url_idx+1)
    imgplot = plt.imshow(image_contents)
    a.set_title(prediction)

plt.show()


## Classify a local image

Previously, we predicted the class of an image based on a URL - what if you have the image as a local file?

Let's download the test images:

In [ ]:
# Get two test images
!wget 'http://www.pachd.com/free-images/food-images/apple-01.jpg' -O "data/test_apple.jpg"
!wget 'http://www.pachd.com/free-images/food-images/carrot-01.jpg' -O "data/test_carrot.jpg"

Now we'll modify our prediction code a little to read the contents of the local image file and submit it to the Custom Vision service.

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

# Use two test images
test_image1_file = os.path.join("data", "test_apple.jpg")
test_image2_file = os.path.join("data", "test_carrot.jpg")

test_images = []
test_images.append(test_image1_file)
test_images.append(test_image2_file)

# Create an instance of the prediction service
predictor = CustomVisionPredictionClient(PREDICTION_KEY, endpoint=ENDPOINT)

# Create a figure
fig = plt.figure(figsize=(16, 8))

# Get the images and show the predicted classes
for idx in range(len(test_images)):
    with open(test_images[idx], "rb") as image_contents:
        results = predictor.classify_image(project.id, iteration.name, image_contents.read())
    # The results include a prediction for each tag, in descending order of probability - so we'll get the first one
    prediction = results.predictions[0].tag_name + ": {0:.2f}%".format(results.predictions[0].probability * 100)
    img = Image.open(test_images[idx])
    # Subplot for image and its predicted class
    a=fig.add_subplot(1,2,idx+1)
    imgplot = plt.imshow(img)
    a.set_title(prediction)

plt.show()


## Learn More
Check out the Custom Vision Service documentation at https://docs.microsoft.com/en-us/azure/cognitive-services/Custom-Vision-Service/home